In [0]:
#Target:
   ## 1.Add rotation 

#Result:
  ## 1:Parameters:9,980
  ## 2.Best train accuracy:98.64
  ## 3.Best test accuracy:99.46(15th epoch)

#Analysis:
   ## 1:The model is working.
   ## 2:The model is underfitting.
   ## 3:we can see 99.4+ repeated more than twice.
   ## 4.The test accuracy increased because of RandomRotation 

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1=nn.Sequential(nn.Conv2d(1,16,3,padding=0,bias=False),        #conv1  1 input , 16 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:3
                                 nn.BatchNorm2d(16),                            #o/p:26
                                 nn.Dropout2d(0.04))
        
        
        self.conv2=nn.Sequential(nn.Conv2d(16,32,3,padding=0,bias=False),       #conv2  16 input , 32 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:5
                                 nn.BatchNorm2d(32),                            #o/p:24
                                 nn.Dropout2d(0.04))
        



        # used 1x1 kernel 
        self.conv3=nn.Sequential(nn.Conv2d(32,8,1,bias=False))                  #conv3  32 input , 8 0utput and 1x1 kernel 
                                                                                #RF:5
                                                                                #o/p:24
        


        self.maxpool=nn.MaxPool2d(2,2)                                          # maxpooling 2x2 kernel and stride 2
                                                                                #RF:6
                                                                                #o/p:12


     

        self.conv4=nn.Sequential(nn.Conv2d(8,14,3,padding=0,bias=False),        #conv4  8 input , 14 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:10
                                 nn.BatchNorm2d(14),                            #o/p:10
                                 nn.Dropout2d(0.04))
                                                                                
        
        self.conv5=nn.Sequential(nn.Conv2d(14,16,3,padding=0,bias=False),       #conv5  14 input , 16 0utput and 3x3 kernel
                                 nn.ReLU(),                                     #RF:14
                                 nn.BatchNorm2d(16),                            #o/p:8
                                 nn.Dropout2d(0.04))                                                
                                                                                
                                                                
        
        #self.maxpool=nn.MaxPool2d(2,2)
                                                               
        self.conv6=nn.Sequential(nn.Conv2d(16,10,1,padding=0,bias=False))       #conv6  16 input , 10 0utput and 3x3 kernel
                                                                                #RF:14
                                                                                #o/p:8
                                                                                

        self.conv7=nn.Sequential(nn.Conv2d(10,16,3,padding=0,bias=False),
                                 nn.ReLU(),                                     #conv6  10 input , 16 0utput and 3x3 kernel
                                 nn.BatchNorm2d(16),                            #RF:18
                                 nn.Dropout2d(0.04))                            #o/p:6           
                                                                                
        
        self.GAP=nn.AvgPool2d(6)                                                # Gobal Average Pooling 
                                                                                #RF:28
                                                                                #o/p:1

                                                                                

        self.conv8 = nn.Sequential(nn.Conv2d(16,10,1, padding=0, bias=False))   #RF:28
                                                                                #O/P:1                                           
                                                                                


             
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        #x=self.maxpool(x)
        x=self.conv3(x)
        x=self.maxpool(x)
        x=self.conv4(x)
        x=self.conv5(x)
        #x=self.maxpool(x)
        x=self.conv6(x)
        x=self.conv7(x)
        x=self.GAP(x)
        x=self.conv8(x)
        x=x.view(-1,10)
        return F.log_softmax(x)


!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,608
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
         Dropout2d-8           [-1, 32, 24, 24]               0
            Conv2d-9            [-1, 8, 24, 24]             256
        MaxPool2d-10            [-1, 8, 12, 12]               0
           Conv2d-11           [-1, 14, 10, 10]           1,008
             ReLU-12           [-1, 14, 10, 10]               0
      BatchNorm2d-13           [-1, 14, 10, 10]              28
        Dropout2d-14           [-1, 14,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:

torch.manual_seed(12)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 16):
    print("epoches:",epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

epoches: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
Loss=0.14838860929012299 Batch_id=468 Accuracy=84.90: 100%|██████████| 469/469 [00:15<00:00, 33.52it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0843, Accuracy: 9790/10000 (97.90%)

epoches: 2


Loss=0.08439373224973679 Batch_id=468 Accuracy=96.64: 100%|██████████| 469/469 [00:15<00:00, 29.32it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0218, Accuracy: 9936/10000 (99.36%)

epoches: 14


Loss=0.02526751160621643 Batch_id=468 Accuracy=98.64: 100%|██████████| 469/469 [00:14<00:00, 32.26it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0207, Accuracy: 9940/10000 (99.40%)

epoches: 15


Loss=0.02337157167494297 Batch_id=468 Accuracy=98.64: 100%|██████████| 469/469 [00:14<00:00, 30.76it/s]



Test set: Average loss: 0.0183, Accuracy: 9946/10000 (99.46%)

